In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import log_loss, confusion_matrix

import math
import matplotlib.pyplot as plt
%matplotlib inline 
from datetime import datetime


In [2]:
import time
import os
import ast
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

import itertools
import re
import string
from collections import Counter

import nltk
from nltk.corpus import stopwords

In [3]:
df = pd.read_csv("fakenews1.csv" )
pd.set_option("display.max_columns",100)
df.tail()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,index,id,author,date,text,title,source,tag,df,tokenized,num_wds,difficulty,uniq_wds,lemmatized,char_count,numbers,mtld,msttr,hdd,avg_word,perc_uniq,avg_sent,caps_body
48527,58100,1.96551e+09,NaN,2016-08-28 00:00:00,28-08-2016 Turkish Bombardment Kills 20 Civili...,Turkish Bombardment Kills 20 Civilians in Syria,manar,1,svdc,28082016 turkish bombardment kills 20 civilian...,240,40,114,28082016 turkish bombardment kill 20 civilian ...,1446,6,13.924646,0.555714,0.403273,5.125000,0.475000,5.125000,0
48528,58101,1.96551e+09,NaN,2016-08-01 00:00:00,17-08-2016 Martyrs as Terrorists Shell Aleppos...,Martyrs as Terrorists Shell Aleppos Salah Eddin,manar,1,svdc,17082016 martyrs as terrorists shell aleppos s...,113,33,64,17082016 martyr a terrorist shell aleppo salah...,726,1,12.476640,0.517143,0.395255,5.504425,0.566372,5.504425,0
48529,58102,1.96551e+09,NaN,2016-08-03 00:00:00,03-08-2016 Chemical Attack Kills Five Syrians ...,Chemical Attack Kills Five Syrians in Aleppo SANA,manar,0,svdc,03082016 chemical attack kills five syrians in...,224,28,130,03082016 chemical attack kill five syrian in a...,1418,2,13.552202,0.539636,0.397377,5.388393,0.580357,5.388393,0
48530,58103,1.96551e+09,NaN,2016-08-01 00:00:00,01-08-2016 5 Killed as Russian Military Choppe...,5 Killed as Russian Military Chopper Shot down...,manar,1,svdc,01082016 5 killed as russian military chopper ...,244,44,121,01082016 5 killed a russian military chopper s...,1483,1,13.401570,0.535862,0.392486,5.159836,0.495902,5.159836,0
48531,58104,1.96551e+09,NaN,2017-04-04 00:00:00,April 6 2017 Syrian Army Kills 48 ISIL Terrori...,Syrian Army Kills 48 ISIL Terrorists in Deir E...,manar,1,svdc,april 6 2017 syrian army kills 48 isil terrori...,408,21,173,april 6 2017 syrian army kill 48 isil terroris...,2565,8,12.903470,0.526800,0.386315,5.366748,0.424020,5.366748,1


In [4]:
df.columns

Index(['index', 'id', 'author', 'date', 'text', 'title', 'source', 'tag', 'df',
       'tokenized', 'num_wds', 'difficulty', 'uniq_wds', 'lemmatized',
       'char_count', 'numbers', 'mtld', 'msttr', 'hdd', 'avg_word',
       'perc_uniq', 'avg_sent', 'caps_body'],
      dtype='object')

## 10 Emotions

In [6]:
filepath = "../../Lexicons/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
emolex_df = pd.read_csv(filepath,  names=["word", "emotion", "association"], skiprows=45, sep='\t')
emolex_df = emolex_df.pivot(index='word', columns='emotion', values='association').reset_index()
emolex_df.head()

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,abacus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
2,abandon,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,abandoned,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,abandonment,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
#CountVectorizer(stop_words='english', binary=True)
# I only want you to look for words in the emotional lexicon
# because we don't know what's up with the other words
vec = TfidfVectorizer(vocabulary=emolex_df.word,
                      use_idf=False, 
                      norm= None ) #'l1') # ELL - ONE
matrix = vec.fit_transform(df['lemmatized'])
vocab = vec.get_feature_names()
wordcount_df = pd.DataFrame(matrix.toarray(), columns=vocab)
wordcount_df.head()

,NaN,abacus,abandon,abandoned,abandonment,abate,abatement,abba,abbot,abbreviate,abbreviation,abdomen,abdominal,abduction,aberrant,aberration,abeyance,abhor,abhorrent,abide,ability,abject,ablation,ablaze,abnormal,aboard,abode,abolish,abolition,abominable,abomination,aboriginal,abort,abortion,abortive,abound,abovementioned,abrasion,abroad,abrogate,abrupt,abruptly,abscess,absence,absent,absentee,absenteeism,absinthe,absolute,absolution,...,yank,yard,yarn,yaw,yawn,yawning,yea,year,yearbook,yearling,yearly,yearn,yearning,years,yeast,yell,yellow,yellows,yelp,yeoman,yesterday,yesteryear,yew,yield,yielding,yogi,yoke,yolk,yon,yonder,young,younger,youth,zany,zap,zeal,zealot,zealous,zebra,zenith,zephyr,zeppelin,zest,zip,zodiac,zone,zoo,zoological,zoology,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
emolex_df.head()

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,abacus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
2,abandon,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,abandoned,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,abandonment,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0


In [10]:
emolex_df[emolex_df.anger == 1].head()

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
3,abandoned,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,abandonment,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
17,abhor,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
18,abhorrent,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
27,abolish,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [11]:
# Get your list of positive words

angry_words = emolex_df[emolex_df.anger == 1]['word']

positive_words = emolex_df[emolex_df.positive == 1]['word']


# Get your list of sadness words
sadness_words = emolex_df[emolex_df.sadness == 1]['word']


# Get your list of surprise words
surprise_words = emolex_df[emolex_df.surprise == 1]['word']


# Get your list of disgust words
disgust_words = emolex_df[emolex_df.disgust == 1]['word']


# Get your list of anticipation words
anticipation_words = emolex_df[emolex_df.anticipation == 1]['word']


# Get your list of negative words
negative_words = emolex_df[emolex_df.negative == 1]['word']



# Get your list of joy words
joy_words = emolex_df[emolex_df.joy == 1]['word']


# Get your list of trust words
trust_words = emolex_df[emolex_df.trust == 1]['word']



# Get your list of fear words
fear_words = emolex_df[emolex_df.fear == 1]['word']




In [12]:
df['anger'] = (wordcount_df[angry_words].sum(axis=1) / df['uniq_wds']) *100

df['positivity'] = (wordcount_df[positive_words].sum(axis=1) / df['uniq_wds']) *100


df['joy'] = (wordcount_df[joy_words].sum(axis=1) / df['uniq_wds']) *100


df['disgust'] = (wordcount_df[disgust_words].sum(axis=1) / df['uniq_wds']) *100



df['surprise'] = (wordcount_df[surprise_words].sum(axis=1) / df['uniq_wds']) *100

df['trust'] = (wordcount_df[trust_words].sum(axis=1) / df['uniq_wds']) *100


df['anticipation'] = (wordcount_df[anticipation_words].sum(axis=1) / df['uniq_wds']) *100


df['sadness'] = (wordcount_df[sadness_words].sum(axis=1) / df['uniq_wds']) *100

df['negative'] = (wordcount_df[negative_words].sum(axis=1) / df['uniq_wds']) *100

df['fear'] = (wordcount_df[fear_words].sum(axis=1) / df['uniq_wds']) *100

df['density'] = df['positivity'] + df['negative']

#df['joysad'] = np.where(df['joy']> df['sadness'], 1, 0)






In [13]:
df['anger'].head(5)

0    12.048193
1     0.684932
2     1.463415
3     4.545455
4     6.024096
Name: anger, dtype: float64

In [14]:
df.head()

,index,id,author,date,text,title,source,tag,df,tokenized,num_wds,difficulty,uniq_wds,lemmatized,char_count,numbers,mtld,msttr,hdd,avg_word,perc_uniq,avg_sent,caps_body,title_polarity,anger,positivity,joy,disgust,surprise,trust,anticipation,sadness,negative,fear,density
0,1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,reasoning with facts,2016-10-29 08:47:11.259000+03:00,Why Did Attorney General Loretta Lynch Plead T...,Re: Why Did Attorney General Loretta Lynch Ple...,100percentfedup.com,bias,grafn,why did attorney general loretta lynch plead t...,277,39,166,why did attorney general loretta lynch plead t...,1732,1,13.459502,0.545882,0.397903,5.447653,0.599278,5.447653,0,0.0000,12.048193,14.457831,4.216867,6.626506,0.602410,13.253012,6.024096,5.421687,16.265060,13.253012,30.722892
1,2,c70e149fdd53de5e61c29281100b9de0ed268bc3,Barracuda Brigade,2016-10-31 01:41:49.479000+02:00,Red State : Fox News Sunday reported this mor...,BREAKING: Weiner Cooperating With FBI On Hilla...,100percentfedup.com,bias,grafn,red state fox news sunday reported this mornin...,222,44,146,red state fox news sunday reported this mornin...,1316,0,12.923091,0.535686,0.382681,5.066964,0.657658,5.066964,0,0.0000,0.684932,8.904110,2.739726,0.684932,0.684932,8.904110,4.109589,0.000000,1.369863,0.684932,10.273973
2,4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,Fed Up,2016-11-01 21:56:00+02:00,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,100percentfedup.com,bias,grafn,email healthcare reform to make america great ...,332,43,205,email healthcare reform to make america great ...,2053,0,13.212205,0.534500,0.393574,5.295181,0.617470,5.295181,6,0.7111,1.463415,10.731707,0.975610,2.926829,1.951220,7.317073,3.902439,3.414634,7.804878,1.951220,18.536585
3,5,8f30f5ea14c9d5914a9fe4f55ab2581772af4c31,Barracuda Brigade,2016-11-02 16:31:28.550000+02:00,Print Hillary goes absolutely berserk! She exp...,Hillary Goes Absolutely Berserk On Protester A...,100percentfedup.com,bias,grafn,print hillary goes absolutely berserk she expl...,178,42,110,print hillary go absolutely berserk she explod...,1174,3,13.387537,0.541739,0.413866,5.836066,0.617978,5.836066,0,0.0000,4.545455,2.727273,0.000000,4.545455,2.727273,2.727273,2.727273,5.454545,14.545455,2.727273,17.272727
4,6,d3cc0fe38f41a59f7c48f8c3528ca5f74193148f,Fed Up,2016-11-04 19:40:00+02:00,BREAKING! NYPD Ready To Make Arrests In Weiner...,BREAKING! NYPD Ready To Make Arrests In Weiner...,100percentfedup.com,bias,grafn,breaking nypd ready to make arrests in weiner ...,955,47,415,breaking nypd ready to make arrest in weiner c...,5630,3,13.120733,0.532613,0.392665,5.080042,0.434555,5.080042,1,0.1759,6.024096,20.963855,5.542169,5.060241,2.409639,14.216867,6.987952,3.614458,10.120482,6.024096,31.084337


## 11 Sentiment

In [15]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

for i in range(df.shape[0]): 
    df.loc[i, "polarity"] = abs(sid.polarity_scores(df.loc[i, 'lemmatized'])['compound'])
    df.loc[i, "negativity_vader"] = abs(sid.polarity_scores(df.loc[i, 'lemmatized'])['neg'])
    df.loc[i, "neutrality_vader"] = abs(sid.polarity_scores(df.loc[i, 'lemmatized'])['neu'])
    df.loc[i, "positivity_vader"] = abs(sid.polarity_scores(df.loc[i, 'lemmatized'])['pos'])



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/medium/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [16]:
## 12 Subjectivity

In [17]:

from textblob import TextBlob, Word

#def detect_polarity(text):
#    return TextBlob(text).sentiment.polarity
def detect_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity


In [18]:
df['subjectivity'] = df.lemmatized.apply(detect_subjectivity)
#df['polarity_textblob'] = df.lemma_content.apply(detect_polarity)

In [19]:
df.tail()

,index,id,author,date,text,title,source,tag,df,tokenized,num_wds,difficulty,uniq_wds,lemmatized,char_count,numbers,mtld,msttr,hdd,avg_word,perc_uniq,avg_sent,caps_body,title_polarity,anger,positivity,joy,disgust,surprise,trust,anticipation,sadness,negative,fear,density,polarity,negativity_vader,neutrality_vader,positivity_vader,subjectivity
48527,58100,1.96551e+09,NaN,2016-08-28 00:00:00,28-08-2016 Turkish Bombardment Kills 20 Civili...,Turkish Bombardment Kills 20 Civilians in Syria,manar,1,svdc,28082016 turkish bombardment kills 20 civilian...,240,40,114,28082016 turkish bombardment kill 20 civilian ...,1446,6,13.924646,0.555714,0.403273,5.125000,0.475000,5.125000,0,NaN,10.526316,0.000000,0.000000,0.000000,0.000000,1.754386,1.754386,2.631579,15.789474,13.157895,15.789474,0.9681,0.126,0.839,0.036,0.213690
48528,58101,1.96551e+09,NaN,2016-08-01 00:00:00,17-08-2016 Martyrs as Terrorists Shell Aleppos...,Martyrs as Terrorists Shell Aleppos Salah Eddin,manar,1,svdc,17082016 martyrs as terrorists shell aleppos s...,113,33,64,17082016 martyr a terrorist shell aleppo salah...,726,1,12.476640,0.517143,0.395255,5.504425,0.566372,5.504425,0,NaN,12.500000,3.125000,1.562500,6.250000,6.250000,1.562500,6.250000,14.062500,20.312500,18.750000,23.437500,0.9861,0.248,0.752,0.000,0.258333
48529,58102,1.96551e+09,NaN,2016-08-03 00:00:00,03-08-2016 Chemical Attack Kills Five Syrians ...,Chemical Attack Kills Five Syrians in Aleppo SANA,manar,0,svdc,03082016 chemical attack kills five syrians in...,224,28,130,03082016 chemical attack kill five syrian in a...,1418,2,13.552202,0.539636,0.397377,5.388393,0.580357,5.388393,0,NaN,9.230769,4.615385,1.538462,2.307692,2.307692,5.384615,3.076923,9.230769,17.692308,16.923077,22.307692,0.9865,0.172,0.803,0.024,0.285000
48530,58103,1.96551e+09,NaN,2016-08-01 00:00:00,01-08-2016 5 Killed as Russian Military Choppe...,5 Killed as Russian Military Chopper Shot down...,manar,1,svdc,01082016 5 killed as russian military chopper ...,244,44,121,01082016 5 killed a russian military chopper s...,1483,1,13.401570,0.535862,0.392486,5.159836,0.495902,5.159836,0,NaN,11.570248,15.702479,4.132231,1.652893,5.785124,13.223140,7.438017,9.917355,9.090909,13.223140,24.793388,0.9744,0.121,0.838,0.042,0.177183
48531,58104,1.96551e+09,NaN,2017-04-04 00:00:00,April 6 2017 Syrian Army Kills 48 ISIL Terrori...,Syrian Army Kills 48 ISIL Terrorists in Deir E...,manar,1,svdc,april 6 2017 syrian army kills 48 isil terrori...,408,21,173,april 6 2017 syrian army kill 48 isil terroris...,2565,8,12.903470,0.526800,0.386315,5.366748,0.424020,5.366748,1,NaN,19.075145,6.358382,3.468208,8.670520,10.404624,5.202312,3.468208,14.450867,21.965318,25.433526,28.323699,0.9991,0.263,0.690,0.047,0.181270


In [39]:
df.columns

Index(['index', 'id', 'author', 'date', 'text', 'title', 'source', 'tag', 'df',
       'tokenized', 'num_wds', 'difficulty', 'uniq_wds', 'lemmatized',
       'char_count', 'numbers', 'mtld', 'msttr', 'hdd', 'avg_word',
       'perc_uniq', 'avg_sent', 'caps_body', 'anger', 'positivity', 'joy',
       'disgust', 'surprise', 'trust', 'anticipation', 'sadness', 'negative',
       'fear', 'density', 'sad', 'happy', 'inspired', 'dont_care', 'annoyed',
       'amused', 'afraid', 'polarity', 'negativity_vader', 'neutrality_vader',
       'positivity_vader', 'subjectivity'],
      dtype='object')

In [40]:
df.to_csv("fakenews2.csv", index = False)